In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-1_z7hd7r/demucs_ea56e496528d4292b0bbb0aa57da07d8
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-1_z7hd7r/demucs_ea56e496528d4292b0bbb0aa57da07d8
  Resolved https://github.com/facebookresearch/demucs to commit e976d93ecc3865e5757426930257e200846a520a
  Preparing metadata (setup.py) ... done


In [3]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 192
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/content/drive/MyDrive/asr/demucs/'
out_path = '/content/drive/MyDrive/asr/demucs_separated/'

In [4]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [5]:
# This can be quite slow, in particular the loading, and saving from GDrive. Please be patient!
# This is from google drive! Also, this will separate all the files inside the MyDrive/demucs folder,
# so when you are happy with the results, remove the songs from there.
separate()

Going to separate the files:
/content/drive/MyDrive/asr/demucs/10945.mp3
/content/drive/MyDrive/asr/demucs/10943.mp3
/content/drive/MyDrive/asr/demucs/10947.mp3
/content/drive/MyDrive/asr/demucs/10949.mp3
/content/drive/MyDrive/asr/demucs/10951.mp3
/content/drive/MyDrive/asr/demucs/10953.mp3
/content/drive/MyDrive/asr/demucs/10957.mp3
/content/drive/MyDrive/asr/demucs/10955.mp3
/content/drive/MyDrive/asr/demucs/10959.mp3
/content/drive/MyDrive/asr/demucs/10961.mp3
/content/drive/MyDrive/asr/demucs/10963.mp3
/content/drive/MyDrive/asr/demucs/10965.mp3
/content/drive/MyDrive/asr/demucs/10967.mp3
/content/drive/MyDrive/asr/demucs/10969.mp3
/content/drive/MyDrive/asr/demucs/10971.mp3
/content/drive/MyDrive/asr/demucs/10973.mp3
/content/drive/MyDrive/asr/demucs/10975.mp3
/content/drive/MyDrive/asr/demucs/10979.mp3
/content/drive/MyDrive/asr/demucs/10981.mp3
/content/drive/MyDrive/asr/demucs/10983.mp3
/content/drive/MyDrive/asr/demucs/10977.mp3
/content/drive/MyDrive/asr/demucs/10985.mp3
/co

100%|██████████████████████████████████████████████| 339.29999999999995/339.29999999999995 [00:14<00:00, 23.05seconds/s]


Separating track /content/drive/MyDrive/asr/demucs/10943.mp3


100%|██████████████████████████████████████████████| 298.34999999999997/298.34999999999997 [00:12<00:00, 24.78seconds/s]


Separating track /content/drive/MyDrive/asr/demucs/10947.mp3


...

100%|████████████████████████████████████████████████████████████████████████| 292.5/292.5 [00:12<00:00, 24.28seconds/s]


Separating track /content/drive/MyDrive/asr/demucs/20729.mp3


100%|██████████████████████████████████████████████| 310.04999999999995/310.04999999999995 [00:12<00:00, 23.85seconds/s]


Separating track /content/drive/MyDrive/asr/demucs/20727.mp3


100%|██████████████████████████████████████████████████████████████████████| 321.75/321.75 [00:13<00:00, 23.49seconds/s]


In [6]:
# # This is manual upload and download :)
# from_upload()
# !zip -r separated.zip separated
# files.download('./separated.zip')